# Import tools

In [6]:
!pip install gingerit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.1 MB/s eta 0:00:00


In [7]:
import numpy as np
import pandas as pd
import os
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random
import spacy
from gingerit.gingerit import GingerIt

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#Reading the text files

In [8]:
txt=[]
from google.colab import files
uploaded_files = files.upload()

for file in uploaded_files.keys():
  with open(file) as f:
    for line in f:
      line = line.strip()
      if line=='----------': break
      if line!='':txt.append(line)

print("number of lines = ", len(txt))



Saving Book 1 - The Philosopher's Stone.txt to Book 1 - The Philosopher's Stone.txt
number of lines =  10144


# Cleaning the text

In [9]:
def clean_txt(txt):
    cleaned_txt = []
    for line in txt:
        line = line.lower()
        line = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "", line)
        tokens = word_tokenize(line)
        words = [word for word in tokens if word.isalpha()]
        cleaned_txt+=words
    return cleaned_txt

cleaned_txt = clean_txt(txt)
print("number of words = ", len(cleaned_txt))

number of words =  83114


## Training the model

In [10]:
def make_markov_model(cleaned_stories, n_gram=2):
    markov_model = {}
    for i in range(len(cleaned_stories)-n_gram-1):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state += cleaned_stories[i+j] + " "
            next_state += cleaned_stories[i+j+n_gram] + " "
        curr_state = curr_state[:-1]
        next_state = next_state[:-1]
        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1

    # calculating transition probabilities
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count/total

    return markov_model

In [11]:
markov_model = make_markov_model(cleaned_txt)
print("number of states = ", len(markov_model.keys()))
print(markov_model.keys())

number of states =  41447
dict_keys(['the boy', 'boy who', 'who lived', 'lived mr', 'mr and', 'and mrs', 'mrs dursley', 'dursley of', 'of number', 'number four', 'four privet', 'privet drive', 'drive were', 'were proud', 'proud to', 'to say', 'say that', 'that they', 'they were', 'were perfectly', 'perfectly normal', 'normal thank', 'thank you', 'you very', 'very much', 'much they', 'were the', 'the last', 'last people', 'people you', 'you d', 'd expect', 'expect to', 'to be', 'be involved', 'involved in', 'in anything', 'anything strange', 'strange or', 'or mysterious', 'mysterious because', 'because they', 'they just', 'just didn', 'didn t', 't hold', 'hold with', 'with such', 'such nonsense', 'nonsense mr', 'mr dursley', 'dursley was', 'was the', 'the director', 'director of', 'of a', 'a firm', 'firm called', 'called grunnings', 'grunnings which', 'which made', 'made drills', 'drills he', 'he was', 'was a', 'a big', 'big beefy', 'beefy man', 'man with', 'with hardly', 'hardly any', 

# Generate Text

In [12]:
def generate_story(markov_model, limit=100, start='my god'):
    n = 0
    curr_state = start
    next_state = None
    story = ""
    story += curr_state + " "
    while n < limit:
        next_state = random.choices(list(markov_model[curr_state].keys()),
                                    list(markov_model[curr_state].values()))

        curr_state = next_state[0]
        story += curr_state + " "
        n += 1
    # story = add_punctuation(story)
    return story

In [13]:
def grammaticize_text(text):
    ginger_parser = GingerIt()
    result = ginger_parser.parse(text)
    return result['result']

In [14]:
def grammaticize_text1(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    # sentences=grammaticize_text(sentences)
    return ".".join(sentences)


In [15]:
story=generate_story(markov_model, start="dark and", limit=30)
result=grammaticize_text1(story)
sent=grammaticize_text(result)
# gramm_story=sent.replace(".\n", ".")
print(sent)

dark and the damp grass reliving the last and youngest of her sister, he didn't feel brave or or any of the students were sitting professor mcgonagall was out of control he was right next to Malfoy here that I've got him out all right before the Muggles that there's still out cold with a little pop he clicked
